In [1]:
import re
import psycopg2
from collections import Counter
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()


True

In [2]:

LOG_FILE = "test_access.log"  # <-- Pointing to test log

ERROR_PATTERN = re.compile(r'(\d{3})\s.*CN=([a-zA-Z0-9.-]+)')

DB_CONFIG = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOSTS"),
    "port": 5432,
}

CREATE_TABLE_HTTP = """
CREATE TABLE IF NOT EXISTS http_status_counts (
    id SERIAL PRIMARY KEY,
    status_code INT NOT NULL,
    count INT NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

CREATE_TABLE_CN = """
CREATE TABLE IF NOT EXISTS cn_counts (
    id SERIAL PRIMARY KEY,
    cn TEXT NOT NULL,
    count INT NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""


In [4]:

def parse_http_statuses(log_file):
    status_pattern = re.compile(r'\s(\d{3})\s')
    counter = Counter()

    with open(log_file, "r") as file:
        for line in file:
            match = status_pattern.search(line)
            if match:
                status_code = match.group(1)
                counter[status_code] += 1

    print("HTTP Status Counts:", counter)
    return counter

def parse_bad_cns(log_file):
    counter = Counter()

    with open(log_file, "r") as file:
        for line in file:
            match = ERROR_PATTERN.search(line)
            if match:
                status_code = match.group(1)
                cn = match.group(2)
                
                if status_code == "400":
                    counter[cn] += 1

    print("CN Counts:", counter)
    return counter

def insert_into_db(status_counts, cn_counts):
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute(CREATE_TABLE_HTTP)
    cur.execute(CREATE_TABLE_CN)

    for status, count in status_counts.items():
        cur.execute(
            "INSERT INTO http_status_counts (status_code, count, timestamp) VALUES (%s, %s, %s);",
            (status, count, datetime.now()),
        )

    for cn, count in cn_counts.items():
        cur.execute(
            "INSERT INTO cn_counts (cn, count, timestamp) VALUES (%s, %s, %s);",
            (cn, count, datetime.now()),
        )

    conn.commit()
    cur.close()
    conn.close()
    print(f"✅ Data inserted at {datetime.now()}")

if __name__ == "__main__":
    http_counts = parse_http_statuses(LOG_FILE)
    cn_counts = parse_bad_cns(LOG_FILE)

    if http_counts or cn_counts:
        insert_into_db(http_counts, cn_counts)
    else:
        print(f"⚠️ No new data found at {datetime.now()}")


HTTP Status Counts: Counter({'400': 30, '200': 15})
CN Counts: Counter({'example1.com': 29, 'idriss.test-home': 1})
✅ Data inserted at 2025-03-18 06:02:41.865164
